In [1]:
import os
import sys
sys.path.append('/'.join(os.getcwd().split('/')[:-1]))
import pandas as pd
pd.set_option('display.precision', 3)
import numpy as np
import datetime
from helpers.dm_test import dm_test

In [2]:
methods = ['MTMFSeq2Seq', 'MTMFSeq2One', 'MLP', 'GBM']

In [3]:
actuals = pd.read_excel(f'../data_electricity/electricity.xlsx',sheet_name='y')
actuals['timestamp'] = pd.to_datetime(actuals['timestamp'])

In [4]:
df_pred = []
for method in methods:
    predictions = pd.read_excel(f'{method}_static/predictions.xlsx',sheet_name='y_prediction')
    df_pred_actual = predictions.loc[predictions['variable_name']=='load_actual'].copy()
    for step_id in [1,2,3,4]:
        df_pred_actual['timestamp'] = df_pred_actual['prev_QE'] + datetime.timedelta(hours=step_id * 6)
        df_pred_actual = pd.merge(df_pred_actual, actuals[['timestamp','load_actual']],on='timestamp',how='left').rename(columns={'load_actual':f'step_{step_id}_actual'})
    df_pred_actual = df_pred_actual.drop(columns=['timestamp']).dropna()
    df_pred_actual['method'] = method
    df_pred.append(df_pred_actual)
df_pred = pd.concat(df_pred)

In [5]:
df_pred['next_24_pred'] = np.exp(df_pred[['step_1','step_2','step_3','step_4']]).sum(axis=1)
df_pred['next_24_actual'] = np.exp(df_pred[['step_1_actual','step_2_actual','step_3_actual','step_4_actual']]).sum(axis=1)

In [6]:
df_error = df_pred[['prev_QE','method','tag']].copy()
for step_id in [1,2,3,4]:
    df_error[f'delta_{step_id}step'] = np.abs(np.exp(df_pred[f'step_{step_id}']) - np.exp(df_pred[f'step_{step_id}_actual']))/np.exp(df_pred[f'step_{step_id}_actual'])
df_error['delta_24'] = np.abs(df_pred['next_24_pred'] - df_pred['next_24_actual'])/df_pred['next_24_actual']

### summarize

In [7]:
df_error_groupby = df_error.groupby(['method','tag']).agg({'delta_24':'mean'})

In [19]:
df_error_pivot = df_error_groupby.pivot_table(index='tag',columns='method',values='delta_24') * 100
df_error_pivot.rename(columns={'GBM':'gbm','MLP':'seq2seq','MTMFSeq2Seq':'seq2one','MTMFSeq2One':'mlp'},inplace=True)
df_error_pivot = df_error_pivot[['seq2seq','seq2one','gbm','mlp']]
df_error_pivot_T = df_error_pivot.T
df_error_pivot_T.index.name = None

In [20]:
df_error_pivot_T

tag,F,N1,N2,N3,N4,N5,N6
seq2seq,4.090,4.091,4.061,4.015,3.962,3.947,3.911
seq2one,4.270,4.286,4.269,4.260,4.255,4.243,4.234
gbm,5.366,5.356,5.358,5.355,5.343,5.328,5.323
mlp,4.391,4.388,4.347,4.312,4.258,4.239,4.218


## Diebold Mariano-Test

In [10]:
df_tag = df_pred.loc[df_pred['tag']=='F',['prev_QE','method','next_24_pred','next_24_actual']].copy().pivot_table(index='prev_QE',columns=['method'],values=['next_24_pred', 'next_24_actual'])

In [11]:
df_tag = df_tag[[('next_24_actual','GBM'), ('next_24_pred','GBM'),('next_24_pred','MLP'),('next_24_pred','MTMFSeq2One'),('next_24_pred', 'MTMFSeq2Seq')]]
df_tag.columns=['actual','GBM','MTMFSeq2Seq','MLP','MTMFSeq2One']

In [12]:
dm_test(df_tag['actual'],df_tag['MTMFSeq2Seq'],df_tag['MTMFSeq2One'],h = 1, crit="MAD")

dm_return(DM=-1.6026643664858773, p_value=0.10944194317121009)

In [13]:
dm_test(df_tag['actual'],df_tag['MTMFSeq2Seq'],df_tag['MLP'],h = 1, crit="MAD")

dm_return(DM=-3.3774325494409867, p_value=0.0007705033342923344)

In [14]:
dm_test(df_tag['actual'],df_tag['MTMFSeq2Seq'],df_tag['GBM'],h = 1, crit="MAD")

dm_return(DM=-10.03969249419394, p_value=2.649952374243309e-22)